InductionCompleted table store information about users who have completed all compulsory trainings. 

It is updated by triger inductionCompleted\_00, on CompletedTrainings Tab

<span style="color: var(--vscode-foreground);">Up to the project requirements Table has to be updated with name, email, project name<br></span>

In [3]:
ALTER TABLE InductionCompleted
ADD Title NVARCHAR(100),
    Email NVARCHAR(100), 
    ProjectName NVARCHAR(100);

: Msg 2705, Level 16, State 4, Line 1
Column names in each table must be unique. Column name 'Title' in table 'InductionCompleted' is specified more than once.

Total execution time: 00:00:00.045

In [19]:
ALTER TRIGGER inductionCompleted_00
ON CompletedTrainings
FOR INSERT AS
BEGIN 
	
	DECLARE @PersonProjectID int
	DECLARE @ProjectID int
    DECLARE @PersonID int
	DECLARE @NumCompletedTrainings int
	DECLARE @NumTrainingsProject int
    DECLARE @Title nvarchar(100)
    DECLARE @Email nvarchar(100) 
    DECLARE @ProjectName nvarchar(100)

	SET @PersonProjectID = (SELECT PersonProjectID FROM inserted)

	SET @ProjectID = (SELECT ProjectID FROM PersonProject WHERE PersonProjectID = @PersonProjectID)
    SET @PersonID = (SELECT PersonID FROM PersonProject WHERE PersonProjectID = @PersonProjectID)

	SET @Title = (SELECT Title FROM People_Details WHERE PersonDetailsID = @PersonID)
    SET @Email = (SELECT Email FROM People_Details WHERE PersonDetailsID = @PersonID) 
    SET @ProjectName = (SELECT ProjectName FROM Projects WHERE ProjectID = @ProjectID)     

	SET @NumTrainingsProject = (SELECT COUNT(*) FROM Trainings WHERE ProjectID = @ProjectID AND Mandatory = 1)
	SET @NumCompletedTrainings = (SELECT COUNT(*) FROM CompletedTrainings ct JOIN Trainings t ON ct.TrainingID = t.TrainingID WHERE t.Mandatory = 1 AND ct.PersonProjectID = @PersonProjectID)

	IF @NumCompletedTrainings >= @NumTrainingsProject
		BEGIN

		INSERT INTO InductionCompleted (PersonProjectId, CompletedDate, Title, Email, ProjectName)
		VALUES (@PersonProjectID, CAST(GETDATE() AS DATE), @Title, @Email, @ProjectName)

		END

END
GO

Commands completed successfully.

Total execution time: 00:00:00.048

<span style="color: rgb(0, 128, 0);">defining groups with people details view for flow</span>

In [5]:
ALTER VIEW PeopleInGroups
AS      
    SELECT pd.Title, pp.PersonProjectID, pp.PersonID, pd.Email, p_g.GroupUrl, p_g.PortalGroupName, p.ProjectName, pg.GroupID
    FROM PersonGroup pg
    JOIN Portal_Groups p_g
    ON pg.GroupID = p_g.PortalGroupID
    JOIN PersonProject pp
    ON pg.PersonProjectID = pp.PersonProjectID
    JOIN People_Details pd 
    ON pp.PersonID = pd.PersonDetailsID
    JOIN Projects p 
    ON pp.ProjectID = p.ProjectID

Commands completed successfully.

Total execution time: 00:00:00.078

In [2]:
ALTER TABLE PersonProject
ADD ProjectRole NVARCHAR(50),
    ProjectWiseLevel NVARCHAR(50)

Commands completed successfully.

Total execution time: 00:00:00.089

Ater Approval Status view

In [8]:
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
ALTER View [dbo].[ApprovalStatus] As
SELECT Pd.PersonDetailsID, pp.PersonProjectID, p.ProjectName as Project, Pd.Email, Pd.Title, Ho.HomeOrganisationName, Iu.Added, Iu.AddedData, Iu.AddedBy
FROM People_Details Pd
FULL OUTER JOIN Home_Organisations Ho
ON Pd.HomeOrganisation = Ho.HomeOrgID
FULL OUTER JOIN PersonProject Pp
ON Pd.PersonDetailsID = Pp.PersonID
FULL OUTER JOIN InfrastructureUsers Iu
ON Pp.PersonProjectID = Iu.PersonProjectID
JOIN Projects p
ON p.ProjectID = Pp.ProjectID
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.135

In [38]:
SELECT Pd.PersonDetailsID, pp.PersonProjectID, p.ProjectName, Pd.Title, Ho.HomeOrganisationName, Iu.Added, Iu.AddedData, Iu.AddedBy
FROM People_Details Pd
FULL OUTER JOIN Home_Organisations Ho
ON Pd.HomeOrganisation = Ho.HomeOrgID
FULL OUTER JOIN PersonProject Pp
ON Pd.PersonDetailsID = Pp.PersonID
FULL OUTER JOIN InfrastructureUsers Iu
ON Pp.PersonProjectID = Iu.PersonProjectID
JOIN Projects p
ON p.ProjectID = Pp.ProjectID
GO

Query was canceled by user

Total execution time: 00:00:48.779

In [2]:
SELECT *
FROM ExternalInfrastructureUsers



(0 rows affected)

Total execution time: 00:00:00.131

Title,Email,HomeOrganisationName,ProjectName,ProjectNumber,AddedBy,AddedData,Comments,EmailText,Added,UserID


In [40]:
DELETE FROM FormsTracker DBCC CHECKIDENT ([FormsTracker], RESEED, 1000)

(1 row affected)

Checking identity information: current identity value '1'.

DBCC execution completed. If DBCC printed error messages, contact your system administrator.

Total execution time: 00:00:00.053

In [41]:
ALTER TABLE People_Details SET (SYSTEM_VERSIONING = OFF);

DELETE FROM People_Details DBCC CHECKIDENT ([People_Details], RESEED, 1000)

ALTER TABLE People_Details SET (SYSTEM_VERSIONING = ON (HISTORY_TABLE = History.People_Details))

(0 rows affected)

Checking identity information: current identity value '0'.

DBCC execution completed. If DBCC printed error messages, contact your system administrator.

Total execution time: 00:00:00.082

In [3]:
CREATE TABLE DestGroups (
    DestGroupID int identity(1000,1) NOT NULL PRIMARY KEY,
    FormTrackID int NOT NULL,
    GroupID int NOT NULL, 
    CONSTRAINT FK_DestGroupForm FOREIGN KEY (FormTrackID)
    REFERENCES FormsTracker(TrackID)
)

Commands completed successfully.

Total execution time: 00:00:00.044

In [8]:
EXECUTE InsertDestGroups_00 
    @FormTrackerID =  1001,
    @GroupIDsList =  '12;15;28';

(3 rows affected)

Total execution time: 00:00:00.062

In [9]:
SELECT *
FROM DestGroups

(3 rows affected)

Total execution time: 00:00:00.041

DestGroupID,FormTrackID,GroupID
1000,1001,12
1001,1001,15
1002,1001,28
